In [1]:
import csv
import sys

# from util import Node, StackFrontier, QueueFrontier

In [2]:
# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

In [3]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, nodelist):
        if type(nodelist)==Node:
            self.frontier.append(nodelist)
        else:
            for node in nodelist:
                if not self.contains_state(node.state):
                    self.add(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def get_node_with_state(self, state):
        if not self.contains_state(state):
            raise Exception("Dont have state {}".format(state))
        for node in self.frontier:
            if node.state == state:
                return node
    
    def state_list(self):
        states = []
        for node in self.frontier:
            states.append(node.state)
        return states
            

    def empty(self):
        return len(self.frontier) == 0
    
    def length(self):
        return len(self.frontier)

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node


In [4]:
def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass
            
def get_movies(actor):
    return list(set(people[actor]['movies']))

def get_actors(mov):
    return list(set(movies[mov]['stars']))

def get_path(node, startNode):
    
    path = []
    while(True):
        if node.parent == startNode.state:
#             print("Done")
            pathlet = (node.action, node.state)
#             print(pathlet)
            path.append((node.action, node.state))
            break
        else:
            pathlet = (node.action, node.state)
#             print(pathlet)
            path.append((node.action, node.state))
            node = checked.get_node_with_state(node.parent)

    path.reverse()
    return path
            
        


In [5]:
load_data("small")

In [ ]:
get_movies('641')

In [ ]:
get_actors('112384')

In [ ]:
get_actors('109830')

In [ ]:
get_actors('109830')

In [ ]:
def get_movies(actor):
    return list(set(people[actor]['movies']))

def get_actors(mov):
    return list(set(movies[mov]['stars']))

In [18]:
source = '1697'
target = '163'
# Frontier = QueueFrontier()
Frontier = StackFrontier()
checked = StackFrontier()
startNode = Node(source,None,None)
Frontier.add(startNode)
foundNode = None
path = []

n=0
while(True):
    removedNode = Frontier.remove()
    checked.add(removedNode)
    removedState = removedNode.state
    print("Iteration {}: Examining actor {}".format(n, removedState))

    theirMovies = get_movies(removedState)
    newNodeList = []
    checkedList = checked.state_list()
    currentList = Frontier.state_list()
    current_actors = Frontier.state_list()

    for mov in theirMovies:
        actors = get_actors(mov)
        actors = [x for x in actors if not x in currentList and not x in checkedList]
        NodeList = [Node(x, removedState, mov) for x in actors]
        newNodeList+=NodeList
    Frontier.add(newNodeList)
    n+=1
    if Frontier.contains_state(target):
        print("success!!!")
        foundNode = Frontier.get_node_with_state(target)
        path = get_path(foundNode,startNode)
        break
    if Frontier.length()<1:
        print("search failed")
        break

print(path)

Iteration 0: Examining actor 1697
Iteration 1: Examining actor 1597
Iteration 2: Examining actor 705
Iteration 3: Examining actor 641
Iteration 4: Examining actor 102
Iteration 5: Examining actor 129
success!!!
('95953', '163')
('104257', '129')
('112384', '102')
('109830', '641')
Done
('93779', '705')
[('93779', '705'), ('109830', '641'), ('112384', '102'), ('104257', '129'), ('95953', '163')]


In [17]:
print(path)
        

[('95953', '129'), ('104257', '102'), ('112384', '641'), ('109830', '705'), ('93779', '1697')]


In [7]:
u=get_path(foundNode,startNode)

('93779', '144')
('109830', '705')
Done
('112384', '641')


In [15]:
people

{'102': {'name': 'Kevin Bacon',
  'birth': '1958',
  'movies': {'104257', '112384'}},
 '129': {'name': 'Tom Cruise', 'birth': '1962', 'movies': {'104257', '95953'}},
 '144': {'name': 'Cary Elwes', 'birth': '1962', 'movies': {'93779'}},
 '158': {'name': 'Tom Hanks', 'birth': '1956', 'movies': {'109830', '112384'}},
 '1597': {'name': 'Mandy Patinkin', 'birth': '1952', 'movies': {'93779'}},
 '163': {'name': 'Dustin Hoffman', 'birth': '1937', 'movies': {'95953'}},
 '1697': {'name': 'Chris Sarandon', 'birth': '1942', 'movies': {'93779'}},
 '193': {'name': 'Demi Moore', 'birth': '1962', 'movies': {'104257'}},
 '197': {'name': 'Jack Nicholson', 'birth': '1937', 'movies': {'104257'}},
 '200': {'name': 'Bill Paxton', 'birth': '1955', 'movies': {'112384'}},
 '398': {'name': 'Sally Field', 'birth': '1946', 'movies': {'109830'}},
 '420': {'name': 'Valeria Golino', 'birth': '1965', 'movies': {'95953'}},
 '596520': {'name': 'Gerald R. Molen', 'birth': '1935', 'movies': {'95953'}},
 '641': {'name': '

In [10]:
get_actors('112384')

['158', '200', '102', '641']

In [12]:
get_actors('109830')

['158', '398', '641', '705']

In [13]:
get_actors('93779')

['144', '1697', '705', '1597']

In [ ]:
def get_path(node):
    path = []
    u = node
    if u.parent== None:
        return []
    for n in range(10):
        print(u.state)
        [v,s]=get_prev_actor(u)
        if v.parent == None:
            print(path)
            return path
        else:
            path.append(s)
            u=v
    print(path)
    return path
        

In [ ]:
path = []
u=foundNode
[v,s]=get_prev_actor(u)
path.append(s)

In [ ]:
[v,s]=get_prev_actor(foundNode)

In [ ]:
p = get_path(foundNode)

In [ ]:
movies['109830']

In [ ]:
v = None

In [ ]:
v is not None

In [ ]:
people

In [ ]:
movies

In [ ]:
pp=[(1,2),(3,4),(5,7)]

In [ ]:
a=(None,None)

In [ ]:
a==(None,None)

In [ ]:
people

In [ ]:
pp = [1,2,3,4,5,6,7]
qq = [x for x in pp if not x<3 and not x%2==0]

In [ ]:
get_movies('129')

In [ ]:
get_actors('95953')

In [ ]:
get_actors('104257')

In [ ]:
get_actors('104257')

In [ ]:
movies

In [ ]:
movies['104257']['stars']